# Homework 1 

### 16:220:630:01 FINANCIAL ECON I

### Team C - Sergei Neznanov, Ruoping Li, Xichen Kong

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import re
import json
import time
import sqlite3
import os
import gc
import networkx as nx
import urllib.request
from itertools import chain
from openpyxl import load_workbook
from collections import Counter

In [ ]:
pip install pyarrow

In [ ]:
pip install fastparquet

# Question 1

Import the data sets for the two days, and the prices:

In [4]:
pd.set_option('display.max_colwidth', None)
infile='trans_tokens_20220701.parquet'
ether_df=pd.read_parquet(infile)
ether_df.head(3)

,block_number,block_timestamp,transaction_hash,transaction_from_address,transaction_to_address,value,receipt_gas_used,token_address,token_from_address,token_to_address,token_value,gas_price
0,15053226,2022-07-01 00:00:00,0x079f98b161166793d2ff41658df69b46625aaf4d2b0bbe13302be7621188f50c,0x46340b20830761efd32832a74d7169b29feb9758,0xdac17f958d2ee523a2206206994597c13d831ec7,0,63209,0xdac17f958d2ee523a2206206994597c13d831ec7,0x46340b20830761efd32832a74d7169b29feb9758,0xd09bba872766ae15fd1eb809659da0d6d937d1e2,2699887308,97937421069
1,15053226,2022-07-01 00:00:00,0x0bf8725a901d0e80e1e71380ae5d256c4041792d98db447ebc3af30ec879fac5,0xf85642d2e867b122f0e1b250c5281b1ef6b26a27,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0,371568,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xcbfb0745b8489973bf7b334d54fdbd573df7ef3c,0xdef1c0ded9bec7f1a1670819833240f027b25eff,22754633,55353921033
2,15053226,2022-07-01 00:00:00,0x0bf8725a901d0e80e1e71380ae5d256c4041792d98db447ebc3af30ec879fac5,0xf85642d2e867b122f0e1b250c5281b1ef6b26a27,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0,371568,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf85642d2e867b122f0e1b250c5281b1ef6b26a27,0xcbfb0745b8489973bf7b334d54fdbd573df7ef3c,4500000000,55353921033


In [5]:
pd.set_option('display.max_colwidth', None)
infile='trans_tokens_20220702.parquet'
ether_df2=pd.read_parquet(infile)
ether_df2.head(3)

,block_number,block_timestamp,transaction_hash,transaction_from_address,transaction_to_address,value,receipt_gas_used,token_address,token_from_address,token_to_address,token_value,gas_price
0,15059681,2022-07-02 00:00:38,0x016cf028d18b86a01c43891692400dad8e71a94ff82716c3a0a165eaf9d13039,0x89e51fa8ca5d66cd220baed62ed01e8951aa7c40,0xdac17f958d2ee523a2206206994597c13d831ec7,0,63209,0xdac17f958d2ee523a2206206994597c13d831ec7,0x89e51fa8ca5d66cd220baed62ed01e8951aa7c40,0x873610638e4f80cb62ac45626fbb7c624b7eaca2,116472200,17354350667
1,15059681,2022-07-02 00:00:38,0x02abeca2f5f6078c54720e65c7df434ca505719e9144789cca93f2145702df3a,0xae2d4617c862309a3d75a0ffb358c7a5009c673f,0xa24787320ede4cc19d800bf87b41ab9539c4da9d,0,71351,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x11c0e4d79f48ce81fbbc69f635e8338afed8c82f,0xae2d4617c862309a3d75a0ffb358c7a5009c673f,100000000000,17354350667
2,15059681,2022-07-02 00:00:38,0x03abf44b5abc4d243d4685d1d2afddc119d9cd977c7acac2dde8526e8212781d,0x1c2ddd05017d2a4bef1a69220b8aa0f9d1c09725,0xdac17f958d2ee523a2206206994597c13d831ec7,0,41321,0xdac17f958d2ee523a2206206994597c13d831ec7,0x1c2ddd05017d2a4bef1a69220b8aa0f9d1c09725,0x28c6c06298d514db089934071355e5743bf21d60,7255072992,17354350667


In [6]:
infile='ETH_20180621_20220708_daily_ticks.csv.gz'
ether_daily_price_df=pd.read_csv(infile)
ether_daily_price_df.rename(columns={'date':'sdate'},inplace=True)
print(ether_daily_price_df.tail(10))
ether_daily_price_df.head()

           sdate  ether_price
1460  2022-06-29      1097.96
1461  2022-06-30      1032.82
1462  2022-07-01      1063.89
1463  2022-07-02      1060.75
1464  2022-07-03      1056.16
1465  2022-07-04      1108.14
1466  2022-07-05      1094.73
1467  2022-07-06      1144.35
1468  2022-07-07      1232.81
1469  2022-07-08      1221.21


,sdate,ether_price
0,2018-06-30,446.38
1,2018-07-01,449.44
2,2018-07-02,471.57
3,2018-07-03,469.93
4,2018-07-04,472.74


Calculate the average price from the output above

In [7]:
ave_price = (1063.89+1060.75)/2
print("The average price is: ", ave_price)


The average price is:  1062.3200000000002


### For assigned token

Group C was assigned token address: 0x6c6ee5e31d828de241282b9606c8e98ea48526e2

In [8]:
details_df=ether_df.query('token_address=="0x6c6ee5e31d828de241282b9606c8e98ea48526e2"')
details_df2=ether_df2.query('token_address=="0x6c6ee5e31d828de241282b9606c8e98ea48526e2"')
details_df.head(3)

,block_number,block_timestamp,transaction_hash,transaction_from_address,transaction_to_address,value,receipt_gas_used,token_address,token_from_address,token_to_address,token_value,gas_price
773,15053230,2022-07-01 00:00:44,0xace79f10aa348e9848397d719b8fa97459f16ae2d3a2773539c148290de67de5,0xdd2300980430e0000000c0c6027ef7a0c93fc6bc,0x0000004f2f05a8223b89edb700848d70970742bf,0,239120,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xa238a470c9121b9a5a6107d7af87eb6bffbe72fe,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,418476381578328935511131,90596005014
1717,15053238,2022-07-01 00:02:52,0x0e0d82bc7d8a370b9393b11b347d34101aad14b376c03d0da1f4ed0fc80d2ea2,0x8e8f818d3371f797a2db7edb32803607c8b3c6a9,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,0,113441,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,369430000000000007177370,69997967014
2976,15053248,2022-07-01 00:05:48,0x7ead28086d2eb6b2c7f177ef175dde1ac0853d9966f988b14e351ebe3da5366b,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0,54453,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0xe0e75a9b83ac7e0b7d78ad10551bf8a1fbca733c,160547686083540000000000,67067847848


### Compute the average number of daily transactions.

In [9]:
total = len(details_df2)+len(details_df)
print("Total number of transactions for Team C token: ", total)

Total number of transactions for Team C token:  549


In [10]:
num_trans_assigned = (len(details_df)+len(details_df2))/2
print("The average number of daily transactions: ", num_trans_assigned)


The average number of daily transactions:  274.5


# Question 2


### Compute the average daily market value in USD of those transactions. You can use prices from CoinMarket Cap for those two days.


On July 1st the closing price of holo was: $0.002036

On July 2nd the closing price of holo was: $0.00205

In [11]:
holo_avg_price = (0.002036 + 0.00205)/2
print(holo_avg_price)

0.002043


In [12]:
details_df['token_value'] = details_df['token_value'].astype('float')
details_df['value'] = (details_df['token_value'] / 10**18)

details_df2['token_value'] = details_df2['token_value'].astype('float')
details_df2['value'] = (details_df2['token_value'] / 10**18)

C:\Users\jessi\AppData\Local\Temp/ipykernel_4480/110378519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_df['token_value'] = details_df['token_value'].astype('float')
C:\Users\jessi\AppData\Local\Temp/ipykernel_4480/110378519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_df['value'] = (details_df['token_value'] / 10**18)
C:\Users\jessi\AppData\Local\Temp/ipykernel_4480/110378519.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [13]:
details_df.head()

,block_number,block_timestamp,transaction_hash,transaction_from_address,transaction_to_address,value,receipt_gas_used,token_address,token_from_address,token_to_address,token_value,gas_price
773,15053230,2022-07-01 00:00:44,0xace79f10aa348e9848397d719b8fa97459f16ae2d3a2773539c148290de67de5,0xdd2300980430e0000000c0c6027ef7a0c93fc6bc,0x0000004f2f05a8223b89edb700848d70970742bf,418476.381578,239120,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xa238a470c9121b9a5a6107d7af87eb6bffbe72fe,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,4.184764e+23,90596005014
1717,15053238,2022-07-01 00:02:52,0x0e0d82bc7d8a370b9393b11b347d34101aad14b376c03d0da1f4ed0fc80d2ea2,0x8e8f818d3371f797a2db7edb32803607c8b3c6a9,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,369430.000000,113441,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,3.694300e+23,69997967014
2976,15053248,2022-07-01 00:05:48,0x7ead28086d2eb6b2c7f177ef175dde1ac0853d9966f988b14e351ebe3da5366b,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,160547.686084,54453,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0xe0e75a9b83ac7e0b7d78ad10551bf8a1fbca733c,1.605477e+23,67067847848
4266,15053266,2022-07-01 00:08:42,0x993dabf2e81a7aa79753293cfb36472a08a6024096ff23b1a2867da1fba5dc9e,0x8e8f818d3371f797a2db7edb32803607c8b3c6a9,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,167344.000000,99805,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xf5ef67632cd2256d939702a126fe2c047d0a07bf,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,1.673440e+23,36401090997
6835,15053294,2022-07-01 00:14:26,0x8db7f0da8b4b424468ab170b369b7a8c9e62f8623943429c25c5558653d6abb2,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,30104.500000,54441,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0x63fde484056de9a112f0185699fe377276ccdbc2,3.010450e+22,29271365847


In [20]:
avg_value = ((details_df.sum()[['value']] * 0.002036) + (details_df2.sum()[['value']] * 0.00205))/2
             
print("The average daily market value in USD of those transactions: ", avg_value)

The average daily market value in USD of those transactions:  value    583212.188649
dtype: object


# Question 3

## On-chain, determine, over the two days:

### (a) the total number of unique senders.


In [11]:
unique_send = details_df['token_from_address'].nunique() + details_df2['token_from_address'].nunique()

In [12]:
print("The total number of unique senders is: ", unique_send)

The total number of unique senders is:  307


### (b) the total number of unique recipients.


In [13]:
unique_rec = details_df['token_to_address'].nunique() + details_df2['token_to_address'].nunique()

In [14]:
print("The total number of unique recipients is: ", unique_rec)

The total number of unique recipients is:  301


### (c) Who are the five most frequent senders based on (i) the number of transactions; (ii) the market value of the transactions. If you can determine the names associated with any address from Etherscan provide it.

In [17]:
details_df['token_from_address'].value_counts().nlargest(5)

0xb7f830845e9f385372ae6fb160aa968908f5bfbc    13
0xf5ef67632cd2256d939702a126fe2c047d0a07bf    10
0x28c6c06298d514db089934071355e5743bf21d60     9
0xdfd5293d8e347dfe59e90efd55b2956a1343963d     9
0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470     9
Name: token_from_address, dtype: int64

In [16]:
details_df2['token_from_address'].value_counts().nlargest(5)

0xf5ef67632cd2256d939702a126fe2c047d0a07bf    19
0x46340b20830761efd32832a74d7169b29feb9758    12
0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21    12
0xdfd5293d8e347dfe59e90efd55b2956a1343963d    11
0xb7f830845e9f385372ae6fb160aa968908f5bfbc    10
Name: token_from_address, dtype: int64

Combine both datasets to find largest combined senders and recipients

In [18]:
concatenated = pd.concat([details_df, details_df2])


In [19]:
top_senders = concatenated['token_from_address'].value_counts().nlargest(5)
print("(i) The top five senders based on the number of transactions: ")
print(top_senders)

(i) The top five senders based on the number of transactions: 
0xf5ef67632cd2256d939702a126fe2c047d0a07bf    29
0xb7f830845e9f385372ae6fb160aa968908f5bfbc    23
0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21    20
0xdfd5293d8e347dfe59e90efd55b2956a1343963d    20
0x28c6c06298d514db089934071355e5743bf21d60    19
Name: token_from_address, dtype: int64


According to Etherscan, these are the names of the following senders:

0xf5ef67632cd2256d939702a126fe2c047d0a07bf   -  No name listed

0xb7f830845e9f385372ae6fb160aa968908f5bfbc   -  No name listed

0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21   -  No name listed

0xdfd5293d8e347dfe59e90efd55b2956a1343963d   -  No name listed

0x28c6c06298d514db089934071355e5743bf21d60   -  No name listed

#### Convert token_value to market value ('value') by dividing by 10^18:

In [20]:
concatenated['token_value'] = concatenated['token_value'].astype('float')
concatenated['value'] = (concatenated['token_value'] / 10**18)

In [65]:
concatenated.head()

,block_number,block_timestamp,transaction_hash,transaction_from_address,transaction_to_address,value,receipt_gas_used,token_address,token_from_address,token_to_address,token_value,gas_price
773,15053230,2022-07-01 00:00:44,0xace79f10aa348e9848397d719b8fa97459f16ae2d3a2773539c148290de67de5,0xdd2300980430e0000000c0c6027ef7a0c93fc6bc,0x0000004f2f05a8223b89edb700848d70970742bf,418476.381578,239120,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xa238a470c9121b9a5a6107d7af87eb6bffbe72fe,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,4.184764e+23,90596005014
1717,15053238,2022-07-01 00:02:52,0x0e0d82bc7d8a370b9393b11b347d34101aad14b376c03d0da1f4ed0fc80d2ea2,0x8e8f818d3371f797a2db7edb32803607c8b3c6a9,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,369430.000000,113441,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,3.694300e+23,69997967014
2976,15053248,2022-07-01 00:05:48,0x7ead28086d2eb6b2c7f177ef175dde1ac0853d9966f988b14e351ebe3da5366b,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,160547.686084,54453,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0x6cc8dcbca746a6e4fdefb98e1d0df903b107fd21,0xe0e75a9b83ac7e0b7d78ad10551bf8a1fbca733c,1.605477e+23,67067847848
4266,15053266,2022-07-01 00:08:42,0x993dabf2e81a7aa79753293cfb36472a08a6024096ff23b1a2867da1fba5dc9e,0x8e8f818d3371f797a2db7edb32803607c8b3c6a9,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,167344.000000,99805,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xf5ef67632cd2256d939702a126fe2c047d0a07bf,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,1.673440e+23,36401090997
6835,15053294,2022-07-01 00:14:26,0x8db7f0da8b4b424468ab170b369b7a8c9e62f8623943429c25c5558653d6abb2,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,30104.500000,54441,0x6c6ee5e31d828de241282b9606c8e98ea48526e2,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0x63fde484056de9a112f0185699fe377276ccdbc2,3.010450e+22,29271365847


In [21]:
top_price_send = concatenated.groupby('token_from_address')(.sum()[['value']]).sort_values(by='value', ascending = False).head(5)
print("(ii) The top five senders based on the market value: ")
print(top_price_send)

(ii) The top five senders based on the market value: 
                                                   value
token_from_address                                      
0x9acbb72cf67103a30333a32cd203459c6a9c3311  1.018905e+08
0x37e1a81bf1a346820836a61af58df637c081d6c7  7.549436e+07
0x95b564f3b3bae3f206aa418667ba000afafacc8a  6.215756e+07
0x29984d1f9055cafb02dcdd53c54b727902e44975  5.964302e+07
0x28c6c06298d514db089934071355e5743bf21d60  4.046808e+07


According to Etherscan, these are the names of the following senders:

0x9acbb72cf67103a30333a32cd203459c6a9c3311  - No name listed

0x37e1a81bf1a346820836a61af58df637c081d6c7  - No name listed

0x95b564f3b3bae3f206aa418667ba000afafacc8a  - No name listed

0x29984d1f9055cafb02dcdd53c54b727902e44975  - No name listed

0x28c6c06298d514db089934071355e5743bf21d60  - No name listed

### (d) Who are the five most frequent recipients based on (i) the number of transactions; (ii) the market value of the transactions. Provide the name as well as the hash if possible 

In [22]:
top_rec = concatenated['token_to_address'].value_counts().nlargest(5)
print("(i) The top five recipients based on the number of transactions: ")
print(top_rec)

(i) The top five recipients based on the number of transactions: 
0x95b564f3b3bae3f206aa418667ba000afafacc8a    126
0xf5ef67632cd2256d939702a126fe2c047d0a07bf     25
0x28c6c06298d514db089934071355e5743bf21d60     18
0xe66b31678d6c16e9ebf358268a790b763c133750     18
0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470     11
Name: token_to_address, dtype: int64


According to Etherscan, these are the names of the following recipients:

0x95b564f3b3bae3f206aa418667ba000afafacc8a - No name listed

0xf5ef67632cd2256d939702a126fe2c047d0a07bf - Uniswap V2 (UNI-V2)

0x28c6c06298d514db089934071355e5743bf21d60 - No name listed

0xe66b31678d6c16e9ebf358268a790b763c133750 - No name listed

0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470 - No name listed

In [23]:
top_price_send = concatenated.groupby('token_to_address').sum()[['value']].sort_values(by='value', ascending = False).head(5)
print("(ii) The top five recipients based on the market value: ")
print(top_price_send)

(ii) The top five recipients based on the market value: 
                                                   value
token_to_address                                        
0x28c6c06298d514db089934071355e5743bf21d60  1.711786e+08
0x37e1a81bf1a346820836a61af58df637c081d6c7  7.549436e+07
0x29984d1f9055cafb02dcdd53c54b727902e44975  5.964302e+07
0x8ecade7f6c5e2996a455fd73661f171301a3c2d6  4.120359e+07
0x39cf16a6bfd99c758b9077270976c288c92747e9  2.001000e+07


According to Etherscan, these are the names of the following recipients:

0x28c6c06298d514db089934071355e5743bf21d60 - No name listed

0x37e1a81bf1a346820836a61af58df637c081d6c7 - No name listed

0x29984d1f9055cafb02dcdd53c54b727902e44975 - No name listed

0x8ecade7f6c5e2996a455fd73661f171301a3c2d6 - No name listed

0x39cf16a6bfd99c758b9077270976c288c92747e9 - No name listed

### (e) Determine the current balance of the most frequent sender and recipient using Web3 (do not cheat and use Etherscan!).


In [72]:
APIKey="5240fae941cd4b168ad1e19c388aa860"
from web3 import Web3
infura_url = "https://mainnet.infura.io/v3/" + APIKey
web3 = Web3(Web3.HTTPProvider(infura_url))
print(web3.isConnected())

True


In [76]:
print("The current balance of the most frequent sender is ")
web3.eth.getBalance('0xf5ef67632cd2256d939702a126FE2c047d0a07bf')

The current balance of the most frequent sender is 


0

Using toChecksumAddress code to fix capitalization in the hashes to access web3

In [79]:
Web3.toChecksumAddress('0x95b564f3b3bae3f206aa418667ba000afafacc8a')

'0x95B564F3B3BaE3f206aa418667bA000AFAFAcc8a'

In [80]:
print("The current balance of the most frequent recipient is ")
web3.eth.getBalance('0x95B564F3B3BaE3f206aa418667bA000AFAFAcc8a')

The current balance of the most frequent recipient is 


4096443391932253660

# 4. Computer the average daily off-chain volumes:

### (a) From Coingecko.com, obtain the average daily volume on the centralized exchange inthe USD pair across all centralized exchanges.

For 7/1/2022, the daily volume was $22,866,439

For 7/2/2022, the daily volume was $21,172,239

The average daily volume was $22,019,339

### (b) From nomics.com, obtain the Coinbase volume for that pair. (Holo isn't listed on Coinbase, evaluating other exchanges)

The volume on Binance for HOT/USDT is $4,371,426